In [ ]:
# Reformat <page> tags to match AU and UK.
import os
import re

# Define session variable
session = '1.July01-July26-1901'

# Define subdirectory path
subdirectory_path = 'txt/hofreps'

# Construct the directory path dynamically
directory = os.path.join(subdirectory_path, session)

def replace_text_in_files(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
            # Replace <page:123> with <page>123</page>
            new_content = re.sub(r'<page:(\d+)>', r'<page>\1</page>', content)
            with open(file_path, 'w', encoding='utf-8') as f:
                f.write(new_content)

# Call the function with the dynamically constructed directory
replace_text_in_files(directory)

In [ ]:
# Documentation, for cell below.

session = '1.July01-July26-1901' 
hathi_id = 'uc1.32106019788238'
starting_page = 29
session = '2.July30-August28-1901' 
hathi_id = 'uc1.32106019788246'
starting_page = 31
session = '3.August29-September25-1901'
hathi_id = 'uc1.32106019788253'
starting_page = 21
session = '4.September26-November07-1901'
hathi_id = 'uc1.32106019788261'
starting_page = 41

In [ ]:
# Correct page numbers so they refer to the Hansard source rather than the Google pdf, and add the corresponding URL for the Hathi online version.

import os
import re

# Define the folder and starting_page
session = '3.August29-September25-1901'
hathi_id = 'uc1.32106019788253'
starting_page = 5
delete_unwanted = 'No'  # Option to delete unwanted content before a certain page

# Construct the path to the subdirectory
subdirectory_path = os.path.join('txt/hofreps', session)

# Initialize the global page number
global_page_number = 1
unwanted_deleted = False

def process_file(file_path, hathi_id, starting_page, delete_unwanted):
    global global_page_number
    global unwanted_deleted

    # Read the content of the file
    with open(file_path, 'r') as file:
        content = file.read()
    
    # If delete_unwanted is set to 'Yes' and unwanted content has not been deleted yet
    if delete_unwanted == 'Yes' and not unwanted_deleted:
        start_index = content.find(f'<page>{starting_page}</page>')
        if start_index != -1:
            content = content[start_index:]
            unwanted_deleted = True
    
    # Find all <page> tags and their corresponding numbers
    page_tags = re.findall(r'<page>(\d+)</page>', content)
    print(f"Found page tags: {page_tags}")
    
    # Split the content by lines
    lines = content.split('\n')
    new_lines = []
    url_added = False
    
    # Iterate over each line and process <page> tags
    for line in lines:
        page_match = re.match(r'<page>(\d+)</page>', line)
        
        if page_match:
            page_number = int(page_match.group(1))
            adjusted_page_number = global_page_number
            global_page_number += 1
            print(f"Original page number: {page_number}, Adjusted page number: {adjusted_page_number}")
            new_lines.append(f'<page>{adjusted_page_number}</page>')
            url = f'<url>https://babel.hathitrust.org/cgi/pt?id={hathi_id}&seq={adjusted_page_number}</url>'
            new_lines.append(url)
            url_added = True
        else:
            if re.match(r'<url>https://babel.hathitrust.org/cgi/pt\?id=.*&seq=\d+</url>', line):
                new_lines.append(line)
                url_added = True
            else:
                new_lines.append(line)
                url_added = False
    
    # Join the new lines to form the updated content
    updated_content = '\n'.join(new_lines)
    
    # Write the updated content back to the file
    with open(file_path, 'w') as file:
        file.write(updated_content)

# Iterate over each file in the subdirectory
for filename in os.listdir(subdirectory_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(subdirectory_path, filename)
        process_file(file_path, hathi_id, starting_page, delete_unwanted)